In [ ]:
#Had a bit of a messy setup. Never used jupyter notebook or .ipynb files before so getting the hang of it.

!pip install fastai

In [ ]:
#I'm going to use this model to detect if an image contains a fire hydrant or not.
#Choosing images with fire hydrants is a common image captcha, and it's interesting how easily this can be beat today.
#Use DuckDuckGo to search for fire hydrant images.

from duckduckgo_search import DDGS
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    with DDGS() as ddgs:
        return [r['image'] for r in ddgs.images(keywords=term, max_results=max_images)]
    
urls = search_images('fire hydrant', max_images=1)
urls[0]

In [ ]:
#Download the image from the found URL. SSL verification was giving me issues so I disabled it.

from fastdownload import download_url
import ssl

# Disable SSL verification
ssl._create_default_https_context = ssl._create_unverified_context

dest = 'firehydrant.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
# Do the same for a reference image. I'll be using forests because they are very unlikely to contain fire hydrants in the background.

urls = search_images('forest', max_images=1)

if urls:
    # Download the image
    dest = 'forest.jpg'  # Changed the filename to reflect the content
    download_url(urls[0], dest, show_progress=False)

    # Open and display the image
    im = Image.open(dest)
    display(im.to_thumb(256,256))
else:
    print("No images found.")

In [ ]:
#Create a dataset of both fire hydrants and forests.

searches = 'fire hydrant','forest'
path = Path('hydrant_or_not')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(o))
    sleep(10)  # Pause between searches to avoid over-loading server
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
#Remove any images that failed to download. Otherwise this will negatively affect our finetune.

failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
#Create a dataloader for our images.

dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

In [ ]:
#Fine-tune the pretrained imagenet model.

learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
#Test the finetune model on our original fire hydrant image.

is_firehydrant,_,probs = learn.predict(PILImage.create('firehydrant.jpg'))
print(f"This is a: {is_firehydrant}.")
print(f"Probability it's a fire hydrant: {probs[0]:.4f}")

In [ ]:
#Save the finetuned model to disk (for fun, not necessary).

learn.export('firehydrant_model.pkl')